## Create large 2D phantom with high number of blobs

This notebook creates a (relatively) large 2D phantom with several 10 of thousands of blobs in it for testing purposes. 

In [8]:
import matplotlib.pyplot as plt
from tnia.simulation.phantoms import random_circles
from tnia.deconvolution import psfs
from tnia.deconvolution import forward
import numpy as np
from tnia.plotting.plt_helper import imshow_multi2d
from pathlib import Path
from skimage.io import imsave

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images\imagesc\2024_01_25_stardist_lots_of_blobs')

## Create Phantom with random circles

In [3]:
load_previous = True
from skimage.io import imread
if load_previous==True:
    img = imread(parent_path / 'img3.tif')
    phantom = imread(parent_path / 'phantom.tif')
else:
    phantom = np.zeros((3789, 3789), dtype=np.float32)
    random_circles(phantom, 40000, 5, 10, 10, 30)
    phantom.min(), phantom.max()

## Create a PSF and apply forward imaging model

The exact parameters don't matter for now.  This will be used to blur the phantom a bit, but the main purpose in this case is just to test memory usage.  (In the future to test performance for different imaging scenarios the blur might be more important)

In [8]:
if load_previous==False:
    n = 200
    wavelength = 500
    na=1.4
    pixel_size = 20

    psf=psfs.paraxial_psf(n, wavelength, na, pixel_size)
    psf.sum()

    img = forward.forward(phantom, psf,10,True, True).astype(np.float32)
    fig = imshow_multi2d([img, phantom], ["img", "phantom"], 1, 2)
    
    imsave(parent_path / 'phantom3.tif', phantom.astype(np.uint16))
    imsave(parent_path / 'img3.tif', img.astype(np.uint16))

1.0

## Inspect images in Napari

In [25]:
import napari
viewer = napari.Viewer()
viewer.add_image(img.astype(np.uint16))
viewer.add_labels(phantom.astype(np.uint16))
viewer.add_labels(labels)

<Labels layer 'labels' at 0x1b9aab190a0>

# Import tensorflow

If we want to test CPU speed, force CPU by setting GPU to not visible.

In [9]:
import tensorflow as tf
visible_devices = tf.config.list_physical_devices()
print(visible_devices)

force_cpu = False

if force_cpu:
    # force CPU
    tf.config.set_visible_devices([], 'GPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
from stardist.models import StarDist2D
from csbdeep.utils import normalize

model = StarDist2D.from_pretrained('2D_versatile_fluo') 
    

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [6]:
normalized_img = normalize(img, 1,99.8, axis=(0,1))

In [7]:
#labels, details = model.predict_instances(img, scale=2, n_tiles=(2,2), prob_thresh=0.1)
labels, details = model.predict_instances(normalized_img)

In [26]:
labels.max()

18068